In [11]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import os
import boto3
import io
import pickle
import boto3
import json

In [10]:
AWS_BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
S3_PREFIX = "documents/"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING_MODEL = "text-embedding-3-small"


S3_CHROMA_KEY = "chroma_memory.json"

In [4]:
print("Working directory:", os.getcwd())

Working directory: c:\Users\yohan\OneDrive\Documents\Yohan\MASTERS\Classes\DAMG 7245\Assignments\Assignment-4\Part-2\DAMG7245_Assignment04_Part02\features\chromadb


In [5]:
chroma_client = chromadb.PersistentClient(path="c:/Users/yohan/OneDrive/Documents/Yohan/MASTERS/Classes/DAMG 7245/Assignments/Assignment-4/Part-2/DAMG7245_Assignment04_Part02/chroma_db")

In [1]:
def download_chroma_from_s3():
    """Download ChromaDB from S3 and recreate collections."""
    chroma_client = chromadb.EphemeralClient()  # Use in-memory ChromaDB

    try:
        response = s3_client.get_object(Bucket=AWS_BUCKET_NAME, Key=S3_CHROMA_KEY)
        collections_data = json.loads(response['Body'].read().decode('utf-8'))
        print("✅ ChromaDB loaded from S3.")

        # Recreate collections
        for collection_name, data in collections_data.items():
            col = chroma_client.get_or_create_collection(collection_name)
            col.add(
                ids=data["ids"],
                documents=data["documents"],
                embeddings=data["embeddings"],
                metadatas=data["metadatas"]
            )

    except s3_client.exceptions.NoSuchKey:
        print("⚠️ No existing ChromaDB found in S3, creating a new one.")
    except Exception as e:
        print(f"⚠️ Error loading ChromaDB: {e}")

In [12]:
base_path = "nvdia/"
s3_client = boto3.client('s3')
try:
    chroma_client = download_chroma_from_s3()
    print("Loaded")
except:
    print("Client not available")

✅ ChromaDB loaded from S3.
Loaded


In [ ]:
def get_embeddings(texts):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )

    return openai_ef(texts)

In [14]:
collection_names = chroma_client.list_collections()
print("Collection names:", collection_names)

AttributeError: 'NoneType' object has no attribute 'list_collections'

In [15]:
print("DEBUGGING 2")
# Access a specific collection by name
collection = chroma_client.get_collection("docling_semantic")
query_text = "What is NVIDIA's approach to AI?"
query_embeddings = get_embeddings([query_text])
results = collection.query(
    query_embeddings=query_embeddings,
    n_results=3,  # Number of results to return
    where={'quarter': 'Q1'}
)

from pprint import pprint
pprint(results)

DEBUGGING 2


AttributeError: 'NoneType' object has no attribute 'get_collection'

## Test-2


In [10]:
import os
import tempfile
import shutil
from s3 import S3FileManager

from typing import List, Dict, Any, Optional
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

import chromadb
import chromadb.utils.embedding_functions as embedding_functions

from dotenv import load_dotenv
load_dotenv()

AWS_BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [11]:
def get_embeddings(texts):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )
    return openai_ef(texts)

In [12]:
def download_chromadb_from_s3(s3_obj, temp_dir):
    """Download ChromaDB files from S3 to a temporary directory"""
    s3_prefix = "chroma_db"
    s3_files = [f for f in s3_obj.list_files() if f.startswith(s3_prefix)]
    
    for s3_file in s3_files:
        # Extract the relative path from the S3 key
        relative_path = s3_file[len(s3_prefix):].lstrip('/')
        local_path = os.path.join(temp_dir, relative_path)
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        
        # Download the file from S3
        content = s3_obj.load_s3_pdf(s3_file)
        with open(local_path, 'wb') as f:
            f.write(content if isinstance(content, bytes) else content.encode('utf-8'))

In [13]:
class QueryRequest(BaseModel):
    query_text: str
    collection_name: str
    n_results: int = 3
    where_filter: Optional[Dict[str, Any]] = None

In [14]:
def query_chromadb(request: QueryRequest):
    # Create a temporary directory
    # with tempfile.TemporaryDirectory() as temp_dir:
    temp_dir = tempfile.mkdtemp()
    try:
        # Initialize S3 connection
        s3_obj = S3FileManager(AWS_BUCKET_NAME, "")
        
        # Download ChromaDB files from S3
        download_chromadb_from_s3(s3_obj, temp_dir)
        
        # Initialize ChromaDB client with the downloaded files
        chroma_client = chromadb.PersistentClient(path=temp_dir)
        
        # Get the requested collection
        try:
            collection = chroma_client.get_collection(request.collection_name)
        except Exception as e:
            raise HTTPException(status_code=404, detail=f"Collection not found: {str(e)}")
        
        # Create embeddings for the query
        query_embeddings = get_embeddings([request.query_text])
        
        # Execute the query
        results = collection.query(
            query_embeddings=query_embeddings,
            n_results=request.n_results,
            where=request.where_filter
        )
        
        return {
            "query": request.query_text,
            "results": results
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error querying ChromaDB: {str(e)}")

In [23]:
where_filter = {
    "$and": [
        {"quarter": {"$eq": "Q1"}},
        {"year": {"$eq": "2025"}}
    ]
}
query_request = QueryRequest(
    query_text="What is NVIDIA's approach to AI?",
    collection_name="mistral_semantic",
    where=where_filter
)

In [24]:
query_chromadb(query_request)

{'query': "What is NVIDIA's approach to AI?",
 'results': {'ids': [['mistral_semantic_FY2025Q1_49',
    'mistral_semantic_FY2025Q3_57',
    'mistral_semantic_FY2025Q1_58']],
  'embeddings': None,
  'documents': [['Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields. Fueled by the sustained demand for exceptional 3D graphics and the scale of the gaming market, NVIDIA has leveraged its GPU architecture to create platforms for scientific computing, AI, data science, autonomous vehicles, robotics, metaverse and 3D internet applications. Our two operating segments are "Compute \\& Networking" and "Graphics," as described in Note 14 of the Notes to Condensed Consolidated Financial Statements in Part I, Item 1 of this Quarterly Report on Form 10-Q.\n\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\n\n ## Recent Developments, F